In [1]:
import os
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

In [2]:
cd F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn

F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
# 图片路径，两组标签都在该目录下
cwd = r".\resize224"
 
# tfrecord文件保存路径
file_path = r"./"



# 每个tfrecord存放图片个数
bestnum = 1000
 
# 第几个图片
num = 0
 
# 第几个TFRecord文件
recordfilenum = 0
 
# 将labels放入到classes中
classes = []
for i in os.listdir(cwd):
    classes.append(i)
print(classes)

# tfrecords格式文件名
ftrecordfilename = ("traindata_cat&dog.tfrecords-%.3d" % recordfilenum)
writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9']


In [7]:
for index, name in enumerate(classes):
    
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num = num + 1
        if num > bestnum:    #超过100，写入下一个tfrecord
            num = 1
            recordfilenum += 1
            ftrecordfilename = ("traindata_cat&dog.tfrecords-%.3d" % recordfilenum)
            writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))
        
        img_path = os.path.join(class_path, img_name)  # 每一个图片的地址
        img = Image.open(img_path, 'r')
        img_raw = img.tobytes()  # 将图片转化为二进制格式
        example = tf.train.Example(
            features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            }))        
        writer.write(example.SerializeToString())  # 序列化为字符串
writer.close()

In [5]:
def read_and_decode_tfrecord(filename):
    filename_deque = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_deque)
    features = tf.parse_single_example(serialized_example, features={
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string)})
    label = tf.cast(features['label'], tf.int32)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])
    img = tf.cast(img, tf.float32) / 255.0      #将矩阵归一化0-1之间
    return img, label

In [9]:
os.listdir()

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'resize224',
 'test224',
 'traindata_cat&dog.tfrecords-000',
 'traindata_cat&dog.tfrecords-001',
 'traindata_cat&dog.tfrecords-002',
 'traindata_cat&dog.tfrecords-003',
 'traindata_cat&dog.tfrecords-004',
 'traindata_cat&dog.tfrecords-005',
 'traindata_cat&dog.tfrecords-006',
 'traindata_cat&dog.tfrecords-007']

In [6]:
train_list = ['traindata_cat&dog.tfrecords-000',
 'traindata_cat&dog.tfrecords-001',
 'traindata_cat&dog.tfrecords-002',
 'traindata_cat&dog.tfrecords-003',
 'traindata_cat&dog.tfrecords-004',
 'traindata_cat&dog.tfrecords-005',
 'traindata_cat&dog.tfrecords-006',
 'traindata_cat&dog.tfrecords-007']

In [7]:
from __future__ import division, print_function, absolute_import

In [8]:
# Training Parameters
learning_rate = 0.001
num_steps = 1
batch_size = 20
display_step = 10

# Network Parameters
num_input = 50176 
num_classes = 37
dropout = 0.75 

# tf Graph input
X = tf.placeholder(tf.float32, [None, 224, 224, 3])
Y = tf.placeholder(tf.float32, [None])
y_ = tf.placeholder(tf.float32, [None,num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [9]:
img, label = read_and_decode_tfrecord(train_list)
img_batch, label_batch = tf.train.shuffle_batch([img, label], num_threads=32, batch_size=batch_size, capacity=10000,min_after_dequeue=9900)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [10]:
# 将label值进行onehot编码
one_hot_labels = tf.one_hot(indices=tf.cast(Y, tf.int32), depth=37)

In [11]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 224, 224, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
    # Convolution Layer
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    
    # Convolution Layer
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = maxpool2d(conv5, k=2)
    
    # Convolution Layer
    conv6 = conv2d(conv5, weights['wc6'], biases['bc6'])
    # Max Pooling (down-sampling)
    conv6 = maxpool2d(conv6, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv6, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)    
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)
    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

In [12]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    # 5x5 conv, 64 inputs, 128 outputs
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    # 5x5 conv, 128 inputs, 256 outputs
    'wc4': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    # 5x5 conv, 128 inputs, 256 outputs
    'wc5': tf.Variable(tf.random_normal([3, 3, 128, 128])),
     # 5x5 conv, 128 inputs, 256 outputs
    'wc6': tf.Variable(tf.random_normal([3, 3, 128, 128])),
    # fully connected, 14*14*256 inputs, 8000 outputs
    'wd1': tf.Variable(tf.random_normal([3*3*128, 512])),
    # fully connected, 8000 inputs, 1024 outputs
    'wd2': tf.Variable(tf.random_normal([512, 128])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([128, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([128])),
    'bc5': tf.Variable(tf.random_normal([128])),
    'bc6': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([512])),
    'bd2': tf.Variable(tf.random_normal([128])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
tf.add_to_collection('network-output', prediction)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
sess=tf.Session()
sess.run(init)

In [14]:
# Start training
saver=tf.train.Saver(max_to_keep=1)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)  
counter=0
# Run the initializer
for epoch in range(1,5):
    print("epoch:",epoch)      
    with tf.device('/gpu:0'):
        for batch in range(1,354): #7040 pics
            batch_x, batch_y = sess.run([img_batch, label_batch])             
            batch_y=sess.run(one_hot_labels, feed_dict={Y: batch_y})            
            # Run optimization op (backprop)                
            sess.run(train_op, feed_dict={X: batch_x, y_: batch_y, keep_prob: dropout})
    #if epoch % display_step == 0 or epoch == 1:        
    # Calculate batch loss and accuracy
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                        y_: batch_y,
                                                                     keep_prob: 1.0})
        print("Epoch " + str(epoch) + ", Minibatch Loss= " + \
                    "{:.4f}".format(loss) + ", Training Accuracy= " + \
                    "{:.3f}".format(acc))
        #saver.save(sess, 'my-model', global_step=1) 
print("Optimization Finished!")
coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
epoch: 1


InvalidArgumentError: Input to reshape is a tensor with 40960 values, but the requested shape requires a multiple of 1152
	 [[node Reshape_2 (defined at <ipython-input-11-2880c1380039>:54) ]]

Caused by op 'Reshape_2', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\lib\asyncio\base_ev

In [25]:
test_dir=r'.\test224\0'
    

In [26]:
for pic in os.listdir(test_dir):
    img_path = os.path.join(test_dir, pic)
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img = tf.reshape(img, [1, 224, 224, 3])
    img1 = tf.reshape(img, [1, 224, 224, 3])
    img = tf.cast(img, tf.float32) / 255.0
    b_image, b_image_raw = sess.run([img, img1])
    haha=conv_net(b_image, weights, biases, 1)
    print(sess.run(haha))
    haha_sm=tf.nn.softmax(haha)
    print(sess.run(tf.argmax(haha_sm, 1)))

[[ 1.1224978e+09 -1.5086779e+09 -3.1489600e+08 -1.2351208e+09
  -1.0056557e+08  3.4595024e+08]]
[0]
[[ 4.35874112e+08 -6.15473792e+08  1.02246317e+09 -2.42703181e+09
   2.25912730e+09  3.91942976e+08]]
[4]
[[ 1.51982976e+09 -2.97822080e+09 -5.78761344e+08 -1.35081434e+09
   1.01452896e+08  1.06476864e+09]]
[0]
[[ 4.1944141e+09  4.5431603e+08  1.6535679e+09 -2.9607718e+09
  -1.2934511e+09 -3.9207536e+07]]
[0]
[[ 2.9943242e+09  1.6902309e+09  1.2088511e+09 -1.3515890e+09
   1.1140163e+09  8.6922394e+08]]
[0]
[[ 5.9589688e+09 -2.1321705e+09  2.5744317e+09 -1.6312666e+09
   1.3892052e+09  1.4447441e+09]]
[0]
[[ 1.2096436e+09 -1.4819078e+09  6.7486074e+08  2.3233034e+08
  -1.1868870e+09 -1.2281321e+09]]
[0]
[[ 2.7715231e+09 -3.5328786e+09  2.3519903e+09 -4.9753210e+08
  -1.8764856e+09  1.9974333e+09]]
[0]
[[-1.5862428e+09  2.5651697e+09  2.7662058e+08 -1.6989066e+09
   2.0320064e+09 -1.5913485e+09]]
[1]
[[ 2.9366740e+09 -3.1318351e+09  1.9210194e+09 -2.3041531e+09
  -1.9056223e+09  1.118363